In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Sep  2 10:39:59 2020

@author: c3121
"""


#%%
import tensorflow as tf
import os
import keras
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import torch
#選用編號1的GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [ ]:
#%%
#將diease資料夾中的圖檔調整為大小128*128並全部存到img_1_array的array中
base_path = r'diease'#這裡要改
img_1=[]
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".JPG"):
            filename = os.path.join(root, file)
            file_size = os.path.getsize(filename)
            category_name = os.path.basename(root)
            #print(filename)
            if file_size >= 2048:
                #print(filename)
                im = cv2.imread(filename)
                im=cv2.resize(im,(128,128), interpolation = cv2.INTER_CUBIC)
                img_1.append(im)
img_1_array=np.asarray(img_1)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, SpatialDropout2D, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import RMSprop, SGD
from keras.layers.normalization import BatchNormalization
#自己修的model
model_cnn = Sequential()
from keras.models import load_model   
    #First Convolution
model_cnn.add(Conv2D(96,(3,3),strides=(2,2),input_shape=(128,128,3),padding='same',activation='relu'))  
    #Second Convolution 
model_cnn.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu'))
    #Two Convolution layer 
model_cnn.add(Conv2D(64,(3,3),strides=(1,1),padding='same',activation='relu'))
model_cnn.add(Conv2D(32,(3,3),strides=(1,1),padding='same',activation='relu'))
    #One Transposed Convolution layer and three Convolution layer
model_cnn.add(Conv2DTranspose(32,(3,3),strides=(2,2),padding='same',activation='relu'))
model_cnn.add(Conv2D(32,(3,3),strides=(1,1),padding='same',activation='relu'))
model_cnn.add(Conv2D(32,(3,3),strides=(1,1),padding='same',activation='relu'))
model_cnn.add(Conv2D(32,(3,3),strides=(1,1),padding='same',activation='relu'))
    #Classfication layer
model_cnn.add(Conv2D(3,(3,3),strides=(1,1),padding='same',activation='softmax'))
    

model_cnn.summary()
optimizer = RMSprop(lr=1e-4)
objective = 'binary_crossentropy'

model_cnn.compile(loss=objective, optimizer=optimizer, metrics=['accuracy'])

In [ ]:
#原始的AlexNet 單純放上來比較 輸出為其layer配置
model_Alex = Sequential()
  

 #First Convolution and Pooling layer
model_Alex.add(Conv2D(96,(11,11),strides=(4,4),input_shape=(128,128,3),padding='valid',activation='relu'))
model_Alex.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
  

  
    #Second Convolution and Pooling layer
model_Alex.add(Conv2D(256,(5,5),strides=(1,1),padding='same',activation='relu'))
model_Alex.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    

    #Three Convolution layer and Pooling Layer
model_Alex.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu'))
model_Alex.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu'))
model_Alex.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu'))
model_Alex.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    

    #Fully connection layer
model_Alex.add(Flatten())
model_Alex.add(Dense(4096,activation='relu'))
model_Alex.add(Dropout(0.5))
model_Alex.add(Dense(4096,activation='relu'))
model_Alex.add(Dropout(0.5))
model_Alex.add(Dense(1000,activation='relu'))
model_Alex.add(Dropout(0.5))
   
    #Classfication layer
model_Alex.add(Dense(2,activation='softmax'))
model_Alex.summary()
optimizer = RMSprop(lr=1e-4)
objective = 'binary_crossentropy'

model_Alex.compile(loss=objective, optimizer=optimizer, metrics=['accuracy'])
#print(len(img_train))

In [ ]:
#訓練
history=model_cnn.fit(img_1_array,img_1_array , batch_size=32, epochs=50,shuffle=True, validation_split=0.2)

In [ ]:
#訓練過程可視化
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
#使用model
from keras.models import load_model   
#選用model
#model_cnn=load_model('fcnn.h5')
n=0
for im in img_1_array[:10]:
    plt.subplot(1,3,1)
    plt.imshow(cv2.cvtColor(im,cv2.COLOR_BGR2RGB))
    imOut=im.copy()
    im=im.astype('float32')/255
    im=im.reshape(-1,128,128,3)
    pred=model_cnn.predict(im)
    pred=pred.astype('float32')
    predOut =pred*255
    predOut = predOut.astype(np.uint8)
    plt.subplot(1,3,2)
    plt.imshow(cv2.cvtColor(predOut[0],cv2.COLOR_BGR2RGB))
    #存檔先註解
    #cv2.imwrite('step1_2/cnn_{}.jpg'.format(n),predOut[0])
    for x in  range(128):
        for y in range(128):
            if(pred[0][x][y][1]>0.3):   
                pred[0][x][y]=(0,0,0)
            else:
                imOut[x][y]=(0,0,255)
    #存檔先註解
    #cv2.imwrite('step1_3/cnn_diease_{}.jpg'.format(n),imOut)
    plt.subplot(1,3,3)
    plt.imshow(cv2.cvtColor(imOut,cv2.COLOR_BGR2RGB))               
    plt.show()
    n=n+1